# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [35]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [36]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [37]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [38]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [39]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [40]:
#Create a Keyspace project_dm
try:
    session.execute("Create KEYSPACE if not exists project_dm\
                    with replication = \
                    {'class' : 'SimpleStrategy', 'replication_factor' : 1}")
               
except Exception as e:
    print(e)

#### Set Keyspace

In [41]:
# Set KEYSPACE to the keyspace project_dm
try:
    session.set_keyspace("project_dm")
except exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Creating a table music_session_history which will have following columns with the data types 
1. sessionid with datatype integer
2. itemInSession with datatype integer
3. artist with datatype string
4. song_title with datatype string
5. song_len with datatype decimal

Table is partitioned on sessionid and itemInsession which would serve as the primary key


In [42]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4

# Create table music_session_history with partition on sessionid and itemInSession which can uniquely \
# identify each row
try:
    session.execute("Create table if not exists music_session_history (sessionid int, itemInSession int,artist text,song_title text, \
                    song_len decimal,\
                    PRIMARY KEY(sessionid,itemInSession))")
except Exception as e:
    print(e)

                    

In [43]:
# Set up the CSV file event_datafile_new.csv  
file = 'event_datafile_new.csv'

# Reading values from csv file and inserting into the table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "Insert into music_session_history(sessionid, itemInSession ,artist,song_title,song_len)"
        query = query + "values(%s,%s,%s,%s,%s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        # Type casting performed to convert string to integer and decimals values for required columns 
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [44]:
# SELECT statement to verify the data was entered into the table music_session_history

# Select only required columns instead of '*' so performance is better. Select query is executed and assigned to result_set
try:
    query = "Select artist,song_title,song_len from music_session_history where sessionid=338 and itemInSession=4;"
    result_set = session.execute(query)
    
except Exception as e:
    print(e)

# Printing the required columns from music_session_history
# result set is put into a dataframe and assigned columns and displayed
try:
    df = pd.DataFrame(list(result_set))
    df.columns = ['artist','song_title','song_len']
    print(df)
    
except Exception as e:
    print(e)

      artist                       song_title  song_len
0  Faithless  Music Matters (Mark Knight Dub)  495.3073


##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Creating a table music_user_history which will have following columns with the data types 
1. userid with datatype integer
1. sessionid with datatype integer
2. itemInSession with datatype integer
3. artist with datatype string
4. song_title with datatype string
5. user with datatype string

Table is partitioned on userid and sessionid.ItemInSession has been added as clustering key to sort the songs based on it. 
userid, sessionid, ItemInSession serve as the primary key.



In [45]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

# Create table music_user_history with partition on userid and sessionid and clustering key as itemInSession to sort songs \
# which together can uniquely identify each row
try:
    session.execute("Create table if not exists music_user_history (userid int, sessionid int,itemInSession int,artist text,song_title text,\
                    user text,PRIMARY KEY((userid,sessionid),itemInSession))")
except Exception as e:
    print(e)
                    

In [46]:
# Reading values from the csv and inserting into the cassandra table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "Insert into music_user_history(userid , sessionid ,itemInSession ,artist ,song_title ,user)"
        query = query + "values(%s,%s,%s,%s,%s,%s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        # Type casting performed on columns to be converted to integer and  user column is concatenation of first and last name
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],line[1]+' '+line[4]))

In [47]:
# SELECT statement to verify the data was entered into the table music_user_history

# Select only required columns instead of '*' so performance is better.Select query is executed and assigned to result_set
try:
    query = "Select artist,itemInSession,song_title,user from music_user_history where userid=10 and sessionid=182;"
    result_set = session.execute(query)
    
except Exception as e:
    print(e)

# Printing the required columns from music_user_history
# result set is put into a dataframe and assigned columns and displayed
try:
    df = pd.DataFrame(list(result_set))
    df.columns = ['artist','itemInSession','song_title','user']
    pd.set_option('display.expand_frame_repr',False) # Bring all columns in line in the result
    print(df)
except Exception as e:
    print(e)

              artist  itemInSession                                         song_title         user
0   Down To The Bone              0                                 Keep On Keepin' On  Sylvie Cruz
1       Three Drives              1                                        Greece 2000  Sylvie Cruz
2  Sebastien Tellier              2                                          Kilometer  Sylvie Cruz
3      Lonnie Gordon              3  Catch You Baby (Steve Pitron & Max Sanna Radio...  Sylvie Cruz


##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Creating a table music_user_song which will have following columns with the data types 
1. username with datatype string
2. song_title with datatype string
3. userid with datatype integer

Table is partitioned on song_title and userid which would serve as the primary key.



In [48]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Create table music_user_song with partition on song title and userid which can uniquely identify \
# each row
try:
    session.execute("Create table if not exists music_user_song (username text,song_title text,userid int,\
                    PRIMARY KEY(song_title,userid))")
except Exception as e:
    print(e)

                    

In [49]:
# Reading values from the csv and inserting into the cassandra table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "Insert into music_user_song(username ,song_title, userid)"
        query = query + "values(%s,%s,%s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        # username column populated with concatenation of first and last name
        session.execute(query, (line[1]+' '+line[4],line[9], int(line[10])))

In [50]:
# SELECT statement to verify the data was entered into the table music_user_song

# Select only required columns instead of '*' so performance is better.Select query is executed and assigned to result_set
try:
    query = "Select username from music_user_song where song_title='All Hands Against His Own';"
    result_set = session.execute(query)
except Exception as e:
    print(e)
    
# Printing the required columns from music_user_song
# result set is put into a dataframe and assigned columns and displayed
try:
    df = pd.DataFrame(list(result_set))
    df.columns = ['username']
    print(df)
except Exception as e:
    print(e)

           username
0  Jacqueline Lynch
1      Tegan Levine
2      Sara Johnson


### Drop the tables before closing out the sessions

In [51]:
# Drop table 1 music_session_history
try:
    session.execute("Drop table music_session_history")
except Exception as e:
    print(e)


# Drop table 2 music_user_history
try:
    session.execute("Drop table music_user_history ")
except Exception as e:
    print(e)

# Drop table 3 music_user_song
try:
    session.execute("Drop table music_user_song ")
except Exception as e:
    print(e)
    

### Close the session and cluster connection¶

In [52]:
session.shutdown()
cluster.shutdown()